In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# USER SETTINGS 
INPUT_ROOT = r"1955"             # year folder containing 01, 02, ... 12
OUTPUT_ROOT = r"1955_images"     # where PNGs will be saved
DEFAULT_SHAPE = (512, 1024)      # known shape of your maps
VMIN = -10
VMAX = 10

def read_IDLdat(filepath):
    data = np.fromfile(filepath, dtype=np.float32)
    return data.reshape(DEFAULT_SHAPE)   # (512, Nlon)

def find_dat_files(root):
    files = []
    for r, d, f in os.walk(root):
        for x in f:
            if x.lower().endswith(".dat"):
                files.append(os.path.join(r, x))
    files.sort()
    return files

def convert_all():
    files = find_dat_files(INPUT_ROOT)
    print("Found", len(files), "files.")

    for path in tqdm(files, desc="Converting"):
        try:
            arr = read_IDLdat(path)

            # lat/lon arrays (exactly same as your code)
            nlat, nlon = arr.shape
            lat = np.linspace(90, -90, nlat)
            lon = np.linspace(0, 360, nlon, endpoint=False)

            # build output filename (mirrors folder structure)
            rel = os.path.relpath(path, INPUT_ROOT)
            out_png = os.path.join(OUTPUT_ROOT, os.path.splitext(rel)[0] + ".png")
            os.makedirs(os.path.dirname(out_png), exist_ok=True)

            # plotting — EXACT same code as your single-day map
            plt.figure(figsize=(10, 4))
            plt.imshow(arr, origin='upper',
                       extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                       aspect='auto',
                       vmin=VMIN, vmax=VMAX)
            plt.colorbar(label='B_r (Gauss?)')
            plt.xlabel('Longitude [deg]')
            plt.ylabel('Latitude [deg]')
            plt.title('Sample map: ' + path)
            plt.tight_layout()
            plt.savefig(out_png, dpi=150)
            plt.close()

        except Exception as e:
            print("Error processing", path, ":", e)

    print("Done! Saved images to:", OUTPUT_ROOT)


# Run conversion
convert_all()


Found 365 files.


Converting: 100%|██████████| 365/365 [02:59<00:00,  2.04it/s]

Done! Saved images to: 1955_images


In [7]:
pip install opencv-python pillow

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.8/39.0 MB 1.2 MB/s eta 0:00:31
   - -------------------------------------- 1.0/39.0 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.3/39.0 MB 1.2 MB/s eta 0:00:32
   - -------------------------------------- 1.6/39.0 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.8/39.0 MB 1.3 MB/s eta 0:00:29
   -- ------------------------------------- 2.1/39.0 MB 1.3 MB/s eta 0:00:29
   -- ------------------------------------- 2.6/39.0 MB 1.4 MB/s eta 0:00:27
   -- ------------------------------------- 2.9/39.0 MB 1.4 MB/s eta 0:00:26
   --- ------------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import glob
import cv2
from PIL import Image
from tqdm import tqdm

# USER SETTINGS 
IMAGES_ROOT = r"1955_images"           # folder containing your PNGs
OUTPUT_VIDEO = r"1955_timelapse.mp4"   # output file
FPS = 10                               # frames per second
RESIZE_TO = None                       # (width, height) or None

def find_pngs(root):
    files = glob.glob(os.path.join(root, "**", "*.png"), recursive=True)
    files = sorted(files)
    return files

def main():
    files = find_pngs(IMAGES_ROOT)
    if not files:
        raise SystemExit("No PNG files found!")

    print(f"Found {len(files)} images")

    # Read first frame
    img = Image.open(files[0]).convert("RGB")
    if RESIZE_TO:
        frame_size = RESIZE_TO
        img = img.resize(frame_size)
    else:
        frame_size = img.size  # (width, height)

    width, height = frame_size

    # Initialize video writer (MP4 with H.264)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, FPS, (width, height))

    # Write frames
    for f in tqdm(files, desc="Adding frames"):
        img = Image.open(f).convert("RGB")
        if img.size != frame_size:
            img = img.resize(frame_size)

        frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        video.write(frame)

    video.release()
    print("DONE! Saved video as:", OUTPUT_VIDEO)

if __name__ == "__main__":
    main()


Found 365 images


Adding frames: 100%|██████████| 365/365 [00:15<00:00, 23.53it/s]

DONE! Saved video as: 1955_timelapse.mp4
